# Importing the required libraries

In [ ]:
import sys
sys.path.append('../')
from tensorflow.keras.callbacks import EarlyStopping
from utils import *
from dataset import *
from comparison_model import *
from ranking_model import *

# Preparing the data

In [ ]:
data_folder = "../data/question_1/duels_question_1.csv"

data = prepare_data(data_folder)

shape = 224


### Splitting and formatting the data for the comparison model using the duels data

In [ ]:
image1_array, image2_array, labels = prepare_dataset_arrays("..\data\question_1\Sample_web_green", data, shape)

### Creating the prediction dataset

In [ ]:
X_pred = prepare_prediction_siamese("../data/question_1/ForPrediction/*/*", shape)

### Creating the Training, Validation and Testing datasets with a split of (60%, 20%, 20%)

In [ ]:
(X_train, y_train), (X_valid, y_valid), (X_test, y_test) = prepare_dataset_for_network(image1_array, image2_array, labels)

# Building the siamese network

## Building the model for the comparison between the two pictures

In [ ]:
# Train the siamese network

early_stopping = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)

siamese_comparison_model = comparison_siamese_model((224, 224, 3))

history = siamese_comparison_model.fit(X_train, y_train, batch_size=64, epochs=5, validation_data=(X_valid, y_valid), callbacks=[early_stopping])

### Testing accuracy check

In [ ]:
# Evaluate the model on the test set
test_accuracy(siamese_comparison_model, X_test, y_test)

### Plotting the accuracy metric for the validation and training datasets

In [ ]:
plot_accuracy(history)

### Plotting the loss for the validation and training datasets

In [ ]:
plot_loss(history)

### Plotting some of the results from the prediction on the Prediction dataset

In [ ]:
# predicting_on_dataset(X_pred, siamese_comparison_model)

# Building the ranking model

In [ ]:
y_train, y_valid, y_test = prepare_label_for_ranking(labels)

### Guillaume's model

In [ ]:
meta_network = create_meta_network(224)
meta_network.summary()
history = meta_network.fit(X_train, y_train, batch_size=16, epochs=5, validation_data=(X_valid, y_valid))

In [ ]:
# plot_loss(history)
# plot_accuracy(history)

In [ ]:
# ranking_model = meta_network.get_layer('Scoring_model')
# ranking_model.save_weights('ranking_model_weights.h5')

In [ ]:
ranking_model = create_ranking_network(224)
ranking_model.load_weights("../Result/Ranking_40E/ranking_model_weights_Adam_Unfrozen_40E.h5")

In [ ]:
plot_ranking_predict(ranking_model, X_pred, "prediction.png")